In [1]:
import pandas as pd
import numpy as np
import json
import snowflake.connector as sc

In [2]:
def conn_open(config_path):
    with open(config_path) as json_file:
        conn_option = json.load(json_file)
    params = {
        'account': conn_option['option'][0]['sf_account'],
        'warehouse': conn_option['option'][0]['sf_warehouse'],
        'database': conn_option['option'][0]['sf_database'],
        'user': conn_option['option'][0]['sf_user'],
        'private_key_file': conn_option['option'][0]['private_key'],
        'private_key_file_pwd': conn_option['option'][0]['private_key_pwd']
    }
    conn = sc.connect(**params)
    return conn

def conn_close(connection):
    if connection:
        connection.close()

In [ ]:
param_dic = 'snowflake_key/connection_option.json'
conn = conn_open(param_dic)

# Product

In [4]:
query = f"""SELECT mid, mid_desc, mg2_desc, desc1, desc2
        FROM SMURDS.BI_F_AI_DETAIL_PRODUCT"""

cursor = conn.cursor()
cursor.execute(query)
tupples = cursor.fetchall()

prod = pd.DataFrame(tupples, columns=["mid", "mid_desc", "mg2_desc", "desc1", "desc2"])

conn_close(conn)

In [5]:
prod.head()

,mid,mid_desc,mg2_desc,desc1,desc2
0,61800,FN COLOGNE DESSERT VIOLET BTL 100ML,COLOGNE SPRAY FRES&NATURAL BTL100ML24PCS,PERSONAL CARE,COLOGNE
1,20322,SEDAAP MIE GORENG AYAM BAKAR LIMAU 89Gr,NOODLE SEDAAPMIE GORENG 40PCS 91GR,FOOD,NOODLES
2,1061830,DAIA POWDET CLEAN&FRESH BAG 800GR NPRG,POWDET DAIA BAG 12PCS 800GR NPRG,FABRIC CARE,POWDER DETERGENT
3,1060153,SOKLIN PEMUTIH REGULAR BTL 500ML,BLEACHING SOKLIN BOTTLE 12PCS 500ML,FABRIC CARE,BLEACHING
4,61229,ALE-ALE GUAVA CUP 180ML,FOODRTD ALE-ALE CUP 200ML 24PCS,BEVERAGE,CUP RTD


### Masking MID

In [6]:
prod1 = prod['desc1'].unique()
prod1_map = {'HOUSE HOLD': '10', 'PERSONAL CARE': '11', 'FABRIC CARE': '12', 'FOOD': '13', 'BEVERAGE': '14'}

prod2 = prod[['desc1', 'desc2']].drop_duplicates().sort_values(['desc1', 'desc2'])
prod2['map'] = prod2.groupby('desc1').cumcount()
prod2['map'] += 10
prod2['map'] = prod2['map'].astype(str)
prod2_map = dict(zip(prod2['desc2'], prod2['map']))

mid = prod['mid'].unique().tolist()
rand_mid = np.random.randint(10000, 20000, 3000).tolist()
rand_mid = list(set(rand_mid))
rand_mid = rand_mid[:len(mid)]
rand_mid = [str(i) for i in rand_mid]

mid_map = dict(zip(mid, rand_mid))
prod['map_ph1'] = prod['desc1'].map(prod1_map)
prod['map_ph2'] = prod['desc2'].map(prod2_map)
prod['map_mid'] = prod['mid'].map(mid_map)
prod['mid_masking'] = prod['map_ph1'] + prod['map_ph2'] + prod['map_mid']

prod = prod.drop(columns=['map_ph1', 'map_ph2', 'map_mid'])
prod = prod.rename(columns={'mid': 'real_mid', 'mid_masking': 'mid'})
prod = prod[["real_mid", "mid", "mid_desc", "desc1", "desc2", "mg2_desc"]]

In [7]:
prod.head()

,real_mid,mid,mid_desc,desc1,desc2,mg2_desc
0,61800,111616385,FN COLOGNE DESSERT VIOLET BTL 100ML,PERSONAL CARE,COLOGNE,COLOGNE SPRAY FRES&NATURAL BTL100ML24PCS
1,20322,131216387,SEDAAP MIE GORENG AYAM BAKAR LIMAU 89Gr,FOOD,NOODLES,NOODLE SEDAAPMIE GORENG 40PCS 91GR
2,1061830,121616390,DAIA POWDET CLEAN&FRESH BAG 800GR NPRG,FABRIC CARE,POWDER DETERGENT,POWDET DAIA BAG 12PCS 800GR NPRG
3,1060153,121116393,SOKLIN PEMUTIH REGULAR BTL 500ML,FABRIC CARE,BLEACHING,BLEACHING SOKLIN BOTTLE 12PCS 500ML
4,61229,141016395,ALE-ALE GUAVA CUP 180ML,BEVERAGE,CUP RTD,FOODRTD ALE-ALE CUP 200ML 24PCS


In [8]:
prod.to_pickle('prod.pkl')

# Customer + Order

In [ ]:
sls_list = ["AA31", "C121", "AA12"]

In [ ]:
for i in sls_list:
    conn = conn_open(param_dic)

    query = f"""SELECT kunnr, vkbur, kdgrp, date_id, mid, value_bill, qty_box FROM smurds.bi_f_ai_history_order 
            WHERE vkbur = '{i}'
            AND date_id BETWEEN '2024-10-01' AND '2024-12-31'"""

    cursor = conn.cursor()
    cursor.execute(query)
    tupples = cursor.fetchall()
    transaction = pd.DataFrame(tupples, columns=["customer_id", "sls_office", "customer_group", "date_id", "mid", "value_bill", "qty_box"])
    conn_close(conn)

    order = transaction[['customer_id', 'sls_office', 'mid', 'date_id', 'value_bill', 'qty_box']].copy()
    cust = transaction[['customer_id', 'customer_group', 'sls_office', "date_id"]].copy()

    order = order.drop_duplicates()
    order['customer_id'] = order['customer_id'].astype(str)
    order['mid'] = order['mid'].astype(str)
    order['date_id'] = pd.to_datetime(order['date_id'])
    order['value_bill'] = order['value_bill'].astype(float)
    order['qty_box'] = order['qty_box'].astype(float)

    cust['customer_id'] = cust['customer_id'].astype(str)
    cust['customer_group'] = cust['customer_group'].astype(str)
    cust['sls_office'] = cust['sls_office'].astype(str)
    cust['date_id'] = pd.to_datetime(cust['date_id'])

    cust_group = cust[['customer_id', 'customer_group', 'date_id']].sort_values(['customer_id', 'date_id'])
    cust_group = cust_group.groupby(['customer_id']).tail(1)
    cust_office = cust.groupby(['customer_id', 'sls_office'], as_index=False).agg({'date_id': 'nunique'})
    cust_office = cust_office.sort_values(['customer_id', 'date_id'], ascending=[True, False])
    cust_office = cust_office.groupby(['customer_id']).head(1)

    cust = cust_group[['customer_id', 'customer_group']].merge(cust_office[['customer_id', 'sls_office']], how='left')
    office_dict = {"AA31": "Caringin", "C121": "Tasikmalaya", "AA12": "Garut"}
    
    cust['sls_office'] = cust['sls_office'].map(office_dict)
    group_desc = pd.read_excel('Master Cust Group.xlsx')
    group_desc = group_desc.rename(columns={'kdgrp': 'customer_group', 'ktext': 'customer_group_desc'})
    group_desc['customer_group'] = group_desc['customer_group'].astype(str)
    cust = cust.merge(group_desc, how='left')
    cust = cust[['customer_id', 'customer_group_desc', 'sls_office']]
    cust = cust.rename(columns={'customer_group_desc': 'customer_group'})
    
    # Filter
    cust = cust[~cust['customer_group'].isin(['Claim', 'Depo SMU', 'Dormant (Non Aktif)', 'Join Ventures SMU', 'Others', 'Outlet w/o Disc.', 'Problem'])]
    order = order[order['customer_id'].isin(cust['customer_id'].unique())]
    mid_masking = dict(zip(prod['real_mid'], prod['mid']))

    order['real_mid'] = order['mid']
    order['mid'] = order['mid'].map(mid_masking)
    cust_id = cust['customer_id'].unique().tolist()
    rand_cust = np.random.randint(100, 200000, 60000).tolist()
    rand_cust = list(set(rand_cust))
    rand_cust = rand_cust[:len(cust_id)]
    cust_map = dict(zip(cust_id, rand_cust))
    cust['customer_id_masking'] = cust['customer_id'].map(cust_map)
    cust = cust.rename(columns={'customer_id': 'real_customer_id', 'customer_id_masking': 'customer_id'})
    cust = cust[["real_customer_id", "customer_id", "customer_group", "sls_office"]]
    cust_masking = dict(zip(cust['real_customer_id'], cust['customer_id']))
    order['customer_id'] = order['customer_id'].map(cust_masking)
    order.to_pickle('order_{}.pkl'.format(i))
    cust.to_pickle('cust_{}.pkl'.format(i))
    del(order, cust)